In [ ]:
!pip install wikipedia
import nltk
nltk.download('wordnet')
import nltk
nltk.download('punkt')
import nltk
nltk.download('stopwords')

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=6ec4cf4dd5e36681d3d7d9ab5ff13cef15b68036c25f767203fb732336042e24
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.5 MB/s eta 0:00:00


In [ ]:
!pip install nltk

# processing text

In [ ]:
import re
import wikipedia
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize WordNetLemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
en_stop = set(stopwords.words('english'))

# Define the process_text function
def process_text(document):
    document = re.sub(r'\s+', ' ', document, flags=re.I)  # Remove extra white space from text
    document = re.sub(r'[^a-zA-Z\s]', '', document)  # Remove all non-alphanumeric characters from text
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)  # Remove all single characters from text
    document = document.lower()  # Converting to Lowercase
    # Word tokenization
    tokens = word_tokenize(document)
    lemma_txt = [lemmatizer.lemmatize(word) for word in tokens]
    lemma_no_stop_txt = [word for word in lemma_txt if word not in en_stop]
    # Drop words
    tokens = [word for word in lemma_no_stop_txt if len(word) > 3]
    clean_txt = ' '.join(tokens)
    return clean_txt

# Get summaries of multiple Wikipedia pages
#topics = ["Indian cuisine", "Pakistani cuisine", "Bangladeshi cuisine"]
topics = ["Video game", "Board game", "Role-playing game"]

#topic_summaries = [wikipedia.summary(topic, sentences=100) for topic in topics]
topic_summaries = [wikipedia.page(topic).content for topic in topics]

# Process and print the text for each summary
for i, summary in enumerate(topic_summaries):
    processed_text = process_text(summary)
    print("\nProcessed Text:")
    print(processed_text)

# Combine all processed text into one corpus
corpus = ' '.join([process_text(text) for text in topic_summaries])


Processed Text:
video game computer game electronic game involves interaction user interface input device joystick controller keyboard motion sensing device generate visual feedback display device commonly shown video format television computer monitor flatpanel display touchscreen handheld device virtual reality headset modern video game audiovisual audio complement delivered speaker headphone sometimes also type sensory feedback haptic technology provides tactile sensation video game also allow microphone webcam input ingame chatting livestreaming video game typically categorized according hardware platform traditionally includes arcade video game console game computer game latter also encompasses game online game browser game recently video game industry expanded onto mobile gaming mobile device smartphones tablet computer virtual augmented reality system remote cloud gaming video game also classified wide range genre based style gameplay target audience first video game prototype 

In [ ]:
len(corpus)

66369

# Tokenize words and one hot encoding

In [ ]:
import re
import numpy as np
import tensorflow as tf
import wikipedia
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import SimpleRNN, GRU, LSTM, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
import numpy as np
doc = SimpleDocTemplate("Results.pdf", pagesize=letter)
styles = getSampleStyleSheet()
pdf_components = []

# ***Word Generation***

In [ ]:
# Tokenize words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus])
word_index = tokenizer.word_index
seq_length = 3
sequences = []

# Generate word sequences
for i in range(seq_length, len(corpus.split())):
    seq = corpus.split()[i-seq_length:i]
    sequences.append(seq)

# Create input and output data
sequences = [' '.join(seq) for seq in sequences]
X = tokenizer.texts_to_sequences(sequences)
X = np.array(X)
y = X[:, -1]
X = X[:, :-1]

vocab_size = len(word_index) + 1

# One-hot encode y
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)

# **One RNN Layer**

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense

# Model building
model_rnn = Sequential()
model_rnn.add(Embedding(vocab_size, 50, input_length=seq_length-1))
model_rnn.add(SimpleRNN(128))  # Use SimpleRNN layer
model_rnn.add(Dense(vocab_size, activation='softmax'))

model_rnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_rnn.fit(X, y, epochs=50, verbose=1)


Epoch 1/50
269/269 [==============================] - 7s 18ms/step - loss: 7.2312 - accuracy: 0.0905
Epoch 2/50
269/269 [==============================] - 6s 21ms/step - loss: 6.6365 - accuracy: 0.0945
Epoch 3/50
269/269 [==============================] - 5s 18ms/step - loss: 6.4170 - accuracy: 0.0951
Epoch 4/50
269/269 [==============================] - 4s 16ms/step - loss: 6.1962 - accuracy: 0.0968
Epoch 5/50
269/269 [==============================] - 4s 13ms/step - loss: 5.9277 - accuracy: 0.0986
Epoch 6/50
269/269 [==============================] - 2s 9ms/step - loss: 5.5761 - accuracy: 0.1127
Epoch 7/50
269/269 [==============================] - 2s 9ms/step - loss: 5.1172 - accuracy: 0.1475
Epoch 8/50
269/269 [==============================] - 3s 9ms/step - loss: 4.6135 - accuracy: 0.2034
Epoch 9/50
269/269 [==============================] - 3s 10ms/step - loss: 4.1138 - accuracy: 0.2723
Epoch 10/50
269/269 [==============================] - 3s 13ms/step - loss: 3.6511 - accuracy:

In [ ]:
# Text generation
start_seq = " game"  # Start sequence for text generation
generated_text = start_seq

for i in range(50):  # Generate 50 words
    token_list = tokenizer.texts_to_sequences([generated_text])[0]
    token_list = pad_sequences([token_list], maxlen=seq_length-1, padding='pre')
    predicted_probs = model_rnn.predict(token_list)[0]
    predicted_index = np.argmax(predicted_probs)

    output_word = ""
    for word, index in word_index.items():
        if index == predicted_index:
            output_word = word
            break
    generated_text += " " + output_word

print("\nGenerated Text:")
print(generated_text)

1/1 [==============================] - 0s 30ms/step

Generated Text:
 game played primarily personal computer type flexible hardware system also commonly used light indicator combination imaging purpose computer game also used descriptor type game tend reduce eliminate dice randomizing element game played primarily personal computer type flexible hardware system also commonly used light indicator combination imaging purpose computer game also used


In [ ]:
heading1 = Paragraph(f"word generation: generatre 50 word", styles["Heading1"])
heading2 = Paragraph(f"Model 1 : one RNN Layer", styles["Heading2"])
text = f"RNN + Dense"
para1 = Paragraph(text, styles["Normal"])
text = f"Generated: {generated_text}"
para2 = Paragraph(text, styles["Normal"])
pdf_components += [heading1, Spacer(1, 20),heading2, Spacer(1, 20), para1, Spacer(1, 20), para2, Spacer(1, 20)]


# **3 RNN Layers**

In [ ]:
# Model building
model_rnn = Sequential()
model_rnn.add(Embedding(vocab_size, 50, input_length=seq_length-1))
model_rnn.add(SimpleRNN(64, return_sequences=True))  # Use SimpleRNN layer with 64 units and return sequences
model_rnn.add(SimpleRNN(32, return_sequences=True))  # Another SimpleRNN layer with 32 units and return sequences
model_rnn.add(SimpleRNN(32))  # Yet another SimpleRNN layer with 32 units
model_rnn.add(Dense(vocab_size, activation='softmax'))

model_rnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_rnn.fit(X, y, epochs=50, verbose=1)

Epoch 1/50
269/269 [==============================] - 11s 15ms/step - loss: 7.2775 - accuracy: 0.0908
Epoch 2/50
269/269 [==============================] - 5s 18ms/step - loss: 6.7230 - accuracy: 0.0938
Epoch 3/50
269/269 [==============================] - 4s 15ms/step - loss: 6.5424 - accuracy: 0.0938
Epoch 4/50
269/269 [==============================] - 3s 13ms/step - loss: 6.3928 - accuracy: 0.0938
Epoch 5/50
269/269 [==============================] - 2s 8ms/step - loss: 6.2378 - accuracy: 0.0954
Epoch 6/50
269/269 [==============================] - 3s 11ms/step - loss: 6.0747 - accuracy: 0.1011
Epoch 7/50
269/269 [==============================] - 3s 11ms/step - loss: 5.9102 - accuracy: 0.1089
Epoch 8/50
269/269 [==============================] - 2s 8ms/step - loss: 5.7458 - accuracy: 0.1150
Epoch 9/50
269/269 [==============================] - 3s 13ms/step - loss: 5.5844 - accuracy: 0.1219
Epoch 10/50
269/269 [==============================] - 3s 10ms/step - loss: 5.4246 - accurac

In [ ]:
# Text generation
start_seq = " game"  # Start sequence for text generation
generated_text = start_seq

for i in range(50):  # Generate 50 words
    token_list = tokenizer.texts_to_sequences([generated_text])[0]
    token_list = pad_sequences([token_list], maxlen=seq_length-1, padding='pre')
    predicted_probs = model_rnn.predict(token_list)[0]
    predicted_index = np.argmax(predicted_probs)

    output_word = ""
    for word, index in word_index.items():
        if index == predicted_index:
            output_word = word
            break
    generated_text += " " + output_word

print("\nGenerated Text:")
print(generated_text)

1/1 [==============================] - 0s 54ms/step

Generated Text:
 game like internet united state japan game often brightly painted logo aspect mehen enjoy claimed drawn consistency fired save feature part video game also used india motivation much across form several called medium engine engine reinforcement chinese mobile emphasizing support role since character mean world film early played platform like odyssey


In [ ]:
heading = Paragraph(f"Model 2 : 3 RNN layers with filters size ( 64 , 32 ,32 )", styles["Heading2"])
text = f"RNN + RNN + RNN + Dense"
para1 = Paragraph(text, styles["Normal"])
text = f"Generated: {generated_text}"
para2 = Paragraph(text, styles["Normal"])
pdf_components += [heading, Spacer(1, 20), para1, Spacer(1, 20), para2, Spacer(1, 20)]

# Using GRU Layer

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense

# Model building
model_gru = Sequential()
model_gru.add(Embedding(vocab_size, 50, input_length=seq_length-1))
model_gru.add(GRU(128))  # Use GRU layer
model_gru.add(Dense(vocab_size, activation='softmax'))

model_gru.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_gru.fit(X, y, epochs=50, verbose=1)


Epoch 1/50
269/269 [==============================] - 5s 11ms/step - loss: 7.2593 - accuracy: 0.0927
Epoch 2/50
269/269 [==============================] - 3s 11ms/step - loss: 6.6972 - accuracy: 0.0940
Epoch 3/50
269/269 [==============================] - 4s 15ms/step - loss: 6.5425 - accuracy: 0.0943
Epoch 4/50
269/269 [==============================] - 3s 13ms/step - loss: 6.3905 - accuracy: 0.0948
Epoch 5/50
269/269 [==============================] - 3s 11ms/step - loss: 6.2365 - accuracy: 0.0934
Epoch 6/50
269/269 [==============================] - 3s 11ms/step - loss: 6.0819 - accuracy: 0.0969
Epoch 7/50
269/269 [==============================] - 4s 13ms/step - loss: 5.9074 - accuracy: 0.0978
Epoch 8/50
269/269 [==============================] - 4s 14ms/step - loss: 5.7212 - accuracy: 0.1001
Epoch 9/50
269/269 [==============================] - 3s 11ms/step - loss: 5.5111 - accuracy: 0.1049
Epoch 10/50
269/269 [==============================] - 3s 11ms/step - loss: 5.2821 - accura

In [ ]:
# Text generation
start_seq = " game"  # Start sequence for text generation
generated_text = start_seq

for i in range(50):  # Generate 50 words
    token_list = tokenizer.texts_to_sequences([generated_text])[0]
    token_list = pad_sequences([token_list], maxlen=seq_length-1, padding='pre')
    predicted_probs = model_gru.predict(token_list)[0]
    predicted_index = np.argmax(predicted_probs)

    output_word = ""
    for word, index in word_index.items():
        if index == predicted_index:
            output_word = word
            break
    generated_text += " " + output_word

print("\nGenerated Text:")
print(generated_text)

1/1 [==============================] - 0s 25ms/step

Generated Text:
 game intermediate point level offer save point player create saved game storage medium restart game lose life need stop game restart later time also form passage written reentered title screen give player chance review option number player larp usually larger tabletop roleplaying game also used therapy setting help individual develop behavioral


In [ ]:
heading = Paragraph(f"Model 3", styles["Heading2"])
text = f"GRU + Dense"
para1 = Paragraph(text, styles["Normal"])
text = f"Generated: {generated_text}"
para2 = Paragraph(text, styles["Normal"])
pdf_components += [heading, Spacer(1, 20), para1, Spacer(1, 20), para2, Spacer(1, 20)]

# RNN+GRU

In [ ]:
from keras.models import Model
from keras.layers import Input, Embedding, SimpleRNN, GRU, concatenate, Dense

# Define input shape
input_layer = Input(shape=(seq_length-1,))

# Embedding layer
embedding_layer = Embedding(vocab_size, 50, input_length=seq_length-1)(input_layer)

# RNN and GRU layers
rnn_layer = SimpleRNN(128)(embedding_layer)
gru_layer = GRU(128)(embedding_layer)

# Concatenate outputs from both RNN and GRU layers
concat_layer = concatenate([rnn_layer, gru_layer])

# Dense layer for prediction
output_layer = Dense(vocab_size, activation='softmax')(concat_layer)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=50, verbose=1)


Epoch 1/50
269/269 [==============================] - 8s 18ms/step - loss: 7.2220 - accuracy: 0.0913
Epoch 2/50
269/269 [==============================] - 4s 15ms/step - loss: 6.6339 - accuracy: 0.0949
Epoch 3/50
269/269 [==============================] - 5s 19ms/step - loss: 6.3841 - accuracy: 0.0950
Epoch 4/50
269/269 [==============================] - 6s 21ms/step - loss: 6.0846 - accuracy: 0.0984
Epoch 5/50
269/269 [==============================] - 5s 17ms/step - loss: 5.6437 - accuracy: 0.1092
Epoch 6/50
269/269 [==============================] - 6s 21ms/step - loss: 5.0408 - accuracy: 0.1452
Epoch 7/50
269/269 [==============================] - 4s 16ms/step - loss: 4.3126 - accuracy: 0.2330
Epoch 8/50
269/269 [==============================] - 4s 15ms/step - loss: 3.5645 - accuracy: 0.3393
Epoch 9/50
269/269 [==============================] - 6s 22ms/step - loss: 2.8794 - accuracy: 0.4464
Epoch 10/50
269/269 [==============================] - 4s 15ms/step - loss: 2.3332 - accura

In [ ]:
# Text generation
start_seq = " game"  # Start sequence for text generation
generated_text = start_seq

for i in range(50):  # Generate 50 words
    token_list = tokenizer.texts_to_sequences([generated_text])[0]
    token_list = pad_sequences([token_list], maxlen=seq_length-1, padding='pre')
    predicted_probs = model.predict(token_list)[0]
    predicted_index = np.argmax(predicted_probs)

    output_word = ""
    for word, index in word_index.items():
        if index == predicted_index:
            output_word = word
            break
    generated_text += " " + output_word

print("\nGenerated Text:")
print(generated_text)

1/1 [==============================] - 0s 20ms/step

Generated Text:
 game platform considered programmable mean read play multiple game distributed different type medium format including cdrom flash memory card furthermore digital distribution including newer mobile gaming marker readilyavailable lowcost development tool platform game played feature color depth refresh rate frame rate screen resolution combination limitation game platform considered programmable mean read


In [ ]:
heading = Paragraph(f"Model 4", styles["Heading2"])
text = f"RNN + GRU + Dense"
para1 = Paragraph(text, styles["Normal"])
text = f"Generated: {generated_text}"
para2 = Paragraph(text, styles["Normal"])
pdf_components += [heading, Spacer(1, 20), para1, Spacer(1, 20), para2, Spacer(1, 20)]

# LSTM

In [ ]:

# Model building
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length-1))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=50, verbose=1)

Epoch 1/50
269/269 [==============================] - 7s 17ms/step - loss: 7.3074 - accuracy: 0.0926
Epoch 2/50
269/269 [==============================] - 3s 12ms/step - loss: 6.7098 - accuracy: 0.0940
Epoch 3/50
269/269 [==============================] - 3s 12ms/step - loss: 6.5884 - accuracy: 0.0940
Epoch 4/50
269/269 [==============================] - 3s 13ms/step - loss: 6.4755 - accuracy: 0.0950
Epoch 5/50
269/269 [==============================] - 4s 17ms/step - loss: 6.3583 - accuracy: 0.0949
Epoch 6/50
269/269 [==============================] - 3s 12ms/step - loss: 6.2425 - accuracy: 0.0947
Epoch 7/50
269/269 [==============================] - 3s 11ms/step - loss: 6.1204 - accuracy: 0.0948
Epoch 8/50
269/269 [==============================] - 4s 15ms/step - loss: 5.9935 - accuracy: 0.0958
Epoch 9/50
269/269 [==============================] - 7s 25ms/step - loss: 5.8583 - accuracy: 0.0973
Epoch 10/50
269/269 [==============================] - 4s 16ms/step - loss: 5.7146 - accura

In [ ]:
# Text generation
start_seq = " game"  # Start sequence for text generation
generated_text = start_seq

for i in range(50):  # Generate 50 words
    token_list = tokenizer.texts_to_sequences([generated_text])[0]
    token_list = pad_sequences([token_list], maxlen=seq_length-1, padding='pre')
    predicted_probs = model.predict(token_list)[0]
    predicted_index = np.argmax(predicted_probs)

    output_word = ""
    for word, index in word_index.items():
        if index == predicted_index:
            output_word = word
            break
    generated_text += " " + output_word

print("\nGenerated Text:")
print(generated_text)


1/1 [==============================] - 0s 24ms/step

Generated Text:
 game developed game platform support portability still create specialized feature direct control game distribution platform method continue used console manufacturer produce console hardware handheld game console game especially game strong rochester york contains largest collection electronic game strong rochester york contains largest collection electronic game strong rochester york contains largest collection


In [ ]:
heading = Paragraph(f"Model 5", styles["Heading2"])
text = f"LSTM + Dense"
para1 = Paragraph(text, styles["Normal"])
text = f"Generated: {generated_text}"
para2 = Paragraph(text, styles["Normal"])
pdf_components += [heading, Spacer(1, 20), para1, Spacer(1, 20), para2, Spacer(1, 20)]

# Character generation

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize

# Create character-level sequences
chars = sorted(list(set(corpus)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Generate input sequences and the corresponding outputs
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(corpus) - maxlen, step):
    sentences.append(corpus[i:i + maxlen]) # Append the current sequence of max_sequence_len characters to sentences list
    next_chars.append(corpus[i + maxlen])# Append the next character after the current sequence to next_chars list

# Vectorization
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
print(len(chars))

27


In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Text generation
def generate_text(length,model,diversity):
    start_index = np.random.randint(0, len(corpus) - maxlen - 1)
    generated = ''
    sentence = corpus[start_index:start_index + maxlen]
    generated += sentence
    for _ in range(length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
    return generated

# **One RNN layer**

In [ ]:
# Model building with SimpleRNN (RNN)
model_rnn = Sequential()
model_rnn.add(SimpleRNN(256, input_shape=(maxlen, len(chars))))
model_rnn.add(Dense(len(chars), activation='softmax'))

optimizer = Adam(lr=0.001)
model_rnn.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with SimpleRNN (RNN)
model_rnn.fit(x, y, batch_size=64, epochs=100)

Epoch 1/100
346/346 [==============================] - 17s 45ms/step - loss: 2.5680 - accuracy: 0.2521
Epoch 2/100
346/346 [==============================] - 17s 50ms/step - loss: 2.2255 - accuracy: 0.3350
Epoch 3/100
346/346 [==============================] - 18s 51ms/step - loss: 2.1208 - accuracy: 0.3580
Epoch 4/100
346/346 [==============================] - 15s 44ms/step - loss: 2.0630 - accuracy: 0.3726
Epoch 5/100
346/346 [==============================] - 15s 45ms/step - loss: 2.0221 - accuracy: 0.3804
Epoch 6/100
346/346 [==============================] - 15s 45ms/step - loss: 1.9826 - accuracy: 0.3868
Epoch 7/100
346/346 [==============================] - 16s 45ms/step - loss: 1.9408 - accuracy: 0.4033
Epoch 8/100
346/346 [==============================] - 17s 49ms/step - loss: 1.8969 - accuracy: 0.4184
Epoch 9/100
346/346 [==============================] - 15s 44ms/step - loss: 1.8554 - accuracy: 0.4295
Epoch 10/100
346/346 [==============================] - 15s 45ms/step - l

In [ ]:
# Generate text
generated_text = generate_text(5,model_rnn, 0.5)  # Generate a longer text
print('\nGenerated Text:')
print(generated_text)



Generated Text:
ved critical decision direction game producti


In [ ]:
# Generate text
generated_text2 = generate_text(5,model_rnn, 0.5)  # Generate a longer text
print('\nGenerated Text:')
print(generated_text2)


Generated Text:
und license dungeon dragon pathfinder market 


In [ ]:
# Generate text
generated_text3 = generate_text(5,model_rnn, 0.5)  # Generate a longer text
print('\nGenerated Text:')
print(generated_text3)


Generated Text:
 strachey draught game nimrod computer freque


In [ ]:
heading1 = Paragraph(f"Character generation : generate 5 char", styles["Heading1"])
heading2 = Paragraph(f"Model 1 : one RNN Layer", styles["Heading2"])
text = f"RNN + Dense"
para1 = Paragraph(text, styles["Normal"])
text = f"Generated1: {generated_text}"
para2 = Paragraph(text, styles["Normal"])
text = f"Generated2: {generated_text2}"
para3 = Paragraph(text, styles["Normal"])
text = f"Generated3: {generated_text3}"
para4 = Paragraph(text, styles["Normal"])
pdf_components += [heading1, Spacer(1, 20),heading2, Spacer(1, 20), para1, Spacer(1, 20), para2, Spacer(1, 20),para3, Spacer(1, 20),para4, Spacer(1, 20)]

# **3 RNN Layers**

In [ ]:
from keras.layers import SimpleRNN

# Model building with SimpleRNN (RNN)
model_rnn = Sequential()
model_rnn.add(SimpleRNN(256, input_shape=(maxlen, len(chars)), return_sequences=True))
model_rnn.add(SimpleRNN(128, return_sequences=True))
model_rnn.add(SimpleRNN(64))
model_rnn.add(Dense(len(chars), activation='softmax'))

optimizer = Adam(lr=0.001)
model_rnn.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with SimpleRNN (RNN)
model_rnn.fit(x, y, batch_size=64, epochs=100)


Epoch 1/100
346/346 [==============================] - 38s 100ms/step - loss: 2.7061 - accuracy: 0.2119
Epoch 2/100
346/346 [==============================] - 54s 156ms/step - loss: 2.2903 - accuracy: 0.3145
Epoch 3/100
346/346 [==============================] - 43s 123ms/step - loss: 2.0983 - accuracy: 0.3640
Epoch 4/100
346/346 [==============================] - 36s 103ms/step - loss: 1.9650 - accuracy: 0.4026
Epoch 5/100
346/346 [==============================] - 41s 120ms/step - loss: 1.8554 - accuracy: 0.4372
Epoch 6/100
346/346 [==============================] - 37s 108ms/step - loss: 1.7676 - accuracy: 0.4628
Epoch 7/100
346/346 [==============================] - 34s 97ms/step - loss: 1.6903 - accuracy: 0.4910
Epoch 8/100
346/346 [==============================] - 37s 106ms/step - loss: 1.6212 - accuracy: 0.5122
Epoch 9/100
346/346 [==============================] - 36s 105ms/step - loss: 1.5680 - accuracy: 0.5266
Epoch 10/100
346/346 [==============================] - 35s 100ms

In [ ]:
# Generate text
generated_text = generate_text(5,model_rnn, 0.5)  # Generate a longer text
print('\nGenerated Text:')
print(generated_text)


Generated Text:
troller like gamepads joystick console action


In [ ]:
# Generate text
generated_text2 = generate_text(5,model_rnn, 0.5)  # Generate a longer text
print('\nGenerated Text:')
print(generated_text2)


Generated Text:
t industry grown building idea others exists 


In [ ]:
# Generate text
generated_text3 = generate_text(5,model_rnn, 0.5)  # Generate a longer text
print('\nGenerated Text:')
print(generated_text3)


Generated Text:
 name draught player capturing opposing parti


In [ ]:
heading = Paragraph(f"Model 2 : 3 RNN Layers with filters size ( 256 , 128 , 64 )", styles["Heading2"])
text = f"RNN + RNN + Dense"
para1 = Paragraph(text, styles["Normal"])
text = f"Generated1: {generated_text}"
para2 = Paragraph(text, styles["Normal"])
text = f"Generated2: {generated_text2}"
para3 = Paragraph(text, styles["Normal"])
text = f"Generated3: {generated_text3}"
para4 = Paragraph(text, styles["Normal"])
pdf_components += [heading, Spacer(1, 20), para1, Spacer(1, 20), para2, Spacer(1, 20), para3, Spacer(1, 20), para4, Spacer(1, 20)]

# GRU

In [ ]:
# Model building with GRU
model_gru = Sequential()
model_gru.add(GRU(256, input_shape=(maxlen, len(chars))))
model_gru.add(Dense(len(chars), activation='softmax'))

optimizer = Adam(lr=0.001)
model_gru.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with GRU
model_gru.fit(x, y, batch_size=64, epochs=40)

Epoch 1/40
346/346 [==============================] - 64s 176ms/step - loss: 2.5195 - accuracy: 0.2592
Epoch 2/40
346/346 [==============================] - 57s 164ms/step - loss: 2.1140 - accuracy: 0.3540
Epoch 3/40
346/346 [==============================] - 49s 142ms/step - loss: 1.9630 - accuracy: 0.3963
Epoch 4/40
346/346 [==============================] - 52s 150ms/step - loss: 1.8342 - accuracy: 0.4385
Epoch 5/40
346/346 [==============================] - 40s 117ms/step - loss: 1.7107 - accuracy: 0.4823
Epoch 6/40
346/346 [==============================] - 46s 134ms/step - loss: 1.5978 - accuracy: 0.5156
Epoch 7/40
346/346 [==============================] - 48s 137ms/step - loss: 1.4919 - accuracy: 0.5476
Epoch 8/40
346/346 [==============================] - 55s 158ms/step - loss: 1.3895 - accuracy: 0.5786
Epoch 9/40
346/346 [==============================] - 49s 143ms/step - loss: 1.2886 - accuracy: 0.6055
Epoch 10/40
346/346 [==============================] - 44s 127ms/step - l

In [ ]:
# Generate text
generated_text = generate_text(5, model_gru,0.5)  # Generate a longer text
print('\nGenerated Text:')
print(generated_text)


Generated Text:
e dungeon dragon unlicensed version developme


In [ ]:
# Generate text
generated_text2 = generate_text(5, model_gru,0.5)  # Generate a longer text
print('\nGenerated Text:')
print(generated_text2)


Generated Text:
ement computer gaming inperson tabletop porti


In [ ]:
# Generate text
generated_text3 = generate_text(5, model_gru,0.5)  # Generate a longer text
print('\nGenerated Text:')
print(generated_text3)


Generated Text:
ucation would enjoy significant benefit term 


In [ ]:
heading = Paragraph(f"Model 3", styles["Heading2"])
text = f"GRU + Dense"
para1 = Paragraph(text, styles["Normal"])
text = f"Generated1: {generated_text}"
para2 = Paragraph(text, styles["Normal"])
text = f"Generated2: {generated_text2}"
para3 = Paragraph(text, styles["Normal"])
text = f"Generated3: {generated_text3}"
para4 = Paragraph(text, styles["Normal"])
pdf_components += [heading, Spacer(1, 20), para1, Spacer(1, 20), para2, Spacer(1, 20), para3, Spacer(1, 20), para4, Spacer(1, 20)]

# RNN + GRU

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Concatenate, SimpleRNN, GRU
from keras.optimizers import Adam

# Model building with SimpleRNN and GRU with concatenation
input_layer = Input(shape=(maxlen, len(chars)))
rnn_output = SimpleRNN(256)(input_layer)
gru_output = GRU(256)(input_layer)
concatenated = Concatenate()([rnn_output, gru_output])
output_layer = Dense(len(chars), activation='softmax')(concatenated)

model_rnn_gru_concat = Model(inputs=input_layer, outputs=output_layer)

optimizer = Adam(lr=0.001)
model_rnn_gru_concat.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with SimpleRNN and GRU with concatenation
model_rnn_gru_concat.fit(x, y, batch_size=64, epochs=40)


Epoch 1/40
346/346 [==============================] - 78s 215ms/step - loss: 2.4232 - accuracy: 0.2815
Epoch 2/40
346/346 [==============================] - 64s 186ms/step - loss: 2.0730 - accuracy: 0.3659
Epoch 3/40
346/346 [==============================] - 60s 173ms/step - loss: 1.9408 - accuracy: 0.4023
Epoch 4/40
346/346 [==============================] - 58s 167ms/step - loss: 1.8297 - accuracy: 0.4415
Epoch 5/40
346/346 [==============================] - 59s 169ms/step - loss: 1.7303 - accuracy: 0.4700
Epoch 6/40
346/346 [==============================] - 57s 164ms/step - loss: 1.6191 - accuracy: 0.5066
Epoch 7/40
346/346 [==============================] - 56s 163ms/step - loss: 1.5215 - accuracy: 0.5351
Epoch 8/40
346/346 [==============================] - 57s 164ms/step - loss: 1.4207 - accuracy: 0.5653
Epoch 9/40
346/346 [==============================] - 59s 170ms/step - loss: 1.3255 - accuracy: 0.5969
Epoch 10/40
346/346 [==============================] - 56s 162ms/step - l

In [ ]:
# Generate text
generated_text = generate_text(5,model_rnn_gru_concat, 0.5)  # Generate a longer text
print('\nGenerated Text:')
print(generated_text)


Generated Text:
endo switch online offering backward computer


In [ ]:
# Generate text
generated_text2 = generate_text(5,model_rnn_gru_concat, 0.5)  # Generate a longer text
print('\nGenerated Text:')
print(generated_text2)


Generated Text:
vide value education across several component


In [ ]:
# Generate text
generated_text3 = generate_text(5,model_rnn_gru_concat, 0.5)  # Generate a longer text
print('\nGenerated Text:')
print(generated_text3)


Generated Text:
sed sale performance smaller boutique publish


In [ ]:
heading = Paragraph(f"Model 4", styles["Heading2"])
text = f"RNN + GRU + Dense"
para1 = Paragraph(text, styles["Normal"])
text = f"Generated1: {generated_text}"
para2 = Paragraph(text, styles["Normal"])
text = f"Generated2: {generated_text2}"
para3 = Paragraph(text, styles["Normal"])
text = f"Generated3: {generated_text3}"
para4 = Paragraph(text, styles["Normal"])
pdf_components += [heading, Spacer(1, 20), para1, Spacer(1, 20), para2, Spacer(1, 20), para3, Spacer(1, 20), para4, Spacer(1, 20)]

# lstm

In [ ]:
# Model building
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x, y, batch_size=64, epochs=30)

Epoch 1/30
346/346 [==============================] - 66s 182ms/step - loss: 2.7179 - accuracy: 0.1992
Epoch 2/30
346/346 [==============================] - 68s 198ms/step - loss: 2.2524 - accuracy: 0.3239
Epoch 3/30
346/346 [==============================] - 54s 155ms/step - loss: 2.0915 - accuracy: 0.3625
Epoch 4/30
346/346 [==============================] - 54s 155ms/step - loss: 1.9735 - accuracy: 0.3938
Epoch 5/30
346/346 [==============================] - 54s 155ms/step - loss: 1.8705 - accuracy: 0.4267
Epoch 6/30
346/346 [==============================] - 53s 153ms/step - loss: 1.7629 - accuracy: 0.4622
Epoch 7/30
346/346 [==============================] - 54s 157ms/step - loss: 1.6614 - accuracy: 0.4960
Epoch 8/30
346/346 [==============================] - 52s 151ms/step - loss: 1.5593 - accuracy: 0.5274
Epoch 9/30
346/346 [==============================] - 54s 156ms/step - loss: 1.4604 - accuracy: 0.5565
Epoch 10/30
346/346 [==============================] - 53s 154ms/step - l

In [ ]:
# Generate text
generated_text = generate_text(5,model, 0.5)  # Generate a longer text
print('\nGenerated Text:')
print(generated_text)


Generated Text:
 idea addition game facilitator rule interact


In [ ]:
# Generate text
generated_text2 = generate_text(5, model, 0.5)  # Generate a longer text
print('\nGenerated Text:')
print(generated_text2)


Generated Text:
pport claim video game beneficial brain story


In [ ]:
# Generate text
generated_text3 = generate_text(5,model, 0.5)  # Generate a longer text
print('\nGenerated Text:')
print(generated_text3)


Generated Text:
indicator combination imaging purpose control


In [ ]:
heading = Paragraph(f"Model 5", styles["Heading2"])
text = f"LSTM + Dense"
para1 = Paragraph(text, styles["Normal"])
text = f"Generated1: {generated_text}"
para2 = Paragraph(text, styles["Normal"])
text = f"Generated2: {generated_text2}"
para3 = Paragraph(text, styles["Normal"])
text = f"Generated3: {generated_text3}"
para4 = Paragraph(text, styles["Normal"])
pdf_components += [heading, Spacer(1, 20), para1, Spacer(1, 20), para2, Spacer(1, 20), para3, Spacer(1, 20), para4, Spacer(1, 20)]

In [ ]:

# Conclusion
heading_conclusion = Paragraph("Conclusion:", styles["Heading1"])
text_conclusion = (
                   "1. Word Generation:<br/>"
                   "   - Model 1 (RNN + Dense):  83.7% - This model produces relevant text and good prediction.<br/>"
                   "   - Model 2 (RNN + RNN + RNN + Dense): 58.9% -  worest model and it's generated words not relevant and doesn't contain any meaning. <br/>"
                   "   - Model 3 (GRU + Dense):  83.8% <br/>"
                   "   - Model 4 (RNN + GRU + Dense): 83.6% <br/>"
                   "   - Model 5 (LSTM + Dense): 82.5% <br/><br/>"
                   "2. Character Generation:<br/>"
                   "   ** Note: it predict more than one character <br/>"
                   "   - Model 1 (RNN + Dense): 81.26% <br/>"
                   "   - Model 2 (RNN + RNN + RNN + Dense): 79.9% - it's result may be similar for using one layer of RNN <br/>"
                   "   - Model 3 (GRU + Dense): 99.2% <br/>"
                   "   - Model 4 (RNN + GRU + Dense): 98.6% <br/>"
                   "   - Model 5 (LSTM + Dense): 98.9% <br/><br/>"
                   "generation of words or characters depends on the number of words I will generate in the following generation. The fewer words or characters I generate, the better the result will be. <br/>"
                   "When comparing the word generation models, Model 1 (RNN + Dense) achieved the highest accuracy of 83.7%. but for Model 2 (RNN + RNN + RNN + Dense) it give worest accuracy .so we note that when we add more layers we get lowest accuracy.<br/>"
                   "Regarding character generation, both Model 1 (RNN + Dense) and Model 2 (RNN + RNN + RNN + Dense) The accuracies of both models are quite close . <br/>"
                   "In summary, Model 1 (RNN + Dense) are best model for word and character generation<br/>"
                   )

para_conclusion = Paragraph(text_conclusion, styles["Normal"])
pdf_components += [heading_conclusion, Spacer(1, 20), para_conclusion]

# Build PDF
doc.build(pdf_components)